# BUILD FIRST MODEL
## (no data preparation)

### Install packages / import namespaces

In [ ]:
#r "nuget:Microsoft.ML,1.5.0"
using Microsoft.ML;
using Microsoft.ML.Data;

### Load data models from Models.cs file 

In [ ]:
#load "C:\Users\dcost\source\repos\SmartFireAlarm\SmartFireAlarm\Jupyter\Models.csx"

### Initialize the ML context (we need it for building the data and training pipelines)  

In [ ]:
MLContext mlContext = new MLContext(0);

### Load data from csv file into a dataview

In [ ]:
const string DATASET_PATH = "./sensors_data.csv";
IDataView data = mlContext.Data.LoadFromTextFile<ModelInput>(
    path: DATASET_PATH,
    hasHeader: true,
    separatorChar: ',');

### Shuffle and split data to _trainingData_ and _testingData_ by a fraction of 0.2

In [ ]:
var shuffledData = mlContext.Data.ShuffleRows(data, seed: 0);
var split = mlContext.Data.TrainTestSplit(shuffledData, testFraction: 0.2);
var trainingData = split.TrainSet;
var testingData = split.TestSet;

### Model builder pipeline

In [ ]:
var featureColumns = new string[] { "Temperature", "Luminosity", "Infrared" };

var trainingPipeline = mlContext.Transforms.Conversion.MapValueToKey("Label")
    .Append(mlContext.Transforms.Concatenate("Features", featureColumns))
    .Append(mlContext.Transforms.NormalizeMinMax("Features"))
    .Append(mlContext.MulticlassClassification.Trainers.SdcaMaximumEntropy("Label", "Features"))
    .Append(mlContext.Transforms.Conversion.MapKeyToValue("PredictedLabel"));

### Train the model

In [ ]:
var model = trainingPipeline.Fit(trainingData);

In [ ]:
var predictions = model.Transform(testingData);
var metrics = mlContext.MulticlassClassification.Evaluate(predictions, "Label", "Score", "PredictedLabel");

In [ ]:
#r "nuget:Microsoft.Data.Analysis"
using Microsoft.AspNetCore.Html;
using Microsoft.Data.Analysis;

#### Load formatter for metrics

In [ ]:
#load "C:\Users\dcost\source\repos\SmartFireAlarm\SmartFireAlarm\Jupyter\Formatters.cs"
var categories = new string[] { "FlashLight", "Infrared", "Day", "Lighter" };
Formatters.Load(categories);

In [ ]:
metrics

In [ ]:
var sampleData = new ModelInput
{
    Luminosity = 92F,
    Temperature = 22F,
    Infrared = 0F
};

var predictor = mlContext.Model.CreatePredictionEngine<ModelInput, ModelOutput>(model);
var predicted = predictor.Predict(sampleData);

display(p[style: "font-size: 24px"]($"{predicted.PredictedLabel}"));

### Save the model

In [ ]:
mlContext.Model.Save(model, trainingData.Schema, "model.zip");

In [ ]:
#r "nuget:Microsoft.ML.OnnxConverter,0.17.0"

### Save the model in ONNX format 

In [ ]:
using (var stream = System.IO.File.Create("model.onnx"))
{
    mlContext.Model.ConvertToOnnx(model, data, stream);
}